# SQL Alchemy

Interacting with a database is hard; it's hard to figue out all the right queries, it's hard to execute your queries in the right way, it's hard to make sure you don't accidentally introduce a SQL injection vulnerability...

The `sqlalchemy` package abstracts the database away, letting you focus on the objects and their interactions rather than having to worry about the database. It's an **ORM**, which is an abstraction layer that provides the underlying plumbing to connect everything to the database and gives you the methods to load and save objects without having to understand how the database works (well, mostly). And it's database-agnostic, meaning your program will run on whatever DBMS you want it to, including SQL-Lite, MySQL, PostgreSQL, and others.

## High-Level Overview

You will work with your database by creating classes that represent the data you want to save and by managing instances of those classes in a `session`. Every `sqlalchemy` program starts by opening a session and finishes by closing the session (I highly recommend using a [with statement](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) for this).

Once you have the session, you can use it as a starting point to query the database and retrieve objects, to change those objects, and to add and delete objects. This set of operations is usually referred to as CRUD:

* **Create:** make a new object
* **Read:** load one or more objects from the database
* **Update:** change the values of stored objects
* **Delete:** remove objects from the database

SQL Alchemy also gives you easy semantics to perform transactions on the database (where either all your changes take place atomically or none of them take place in the final result).

## Getting Started

First you need to install `sqlalchemy`:

```bash
$ pip install sqlalchemy
Collecting sqlalchemy
  Downloading SQLAlchemy-1.2.1.tar.gz (5.5MB)
    100% |████████████████████████████████| 5.5MB 279kB/s 
Building wheels for collected packages: sqlalchemy
  Running setup.py bdist_wheel for sqlalchemy ... done
  Stored in directory: /Users/seawolf/Library/Caches/pip/wheels/cb/4a/f4/ddc8af56687863cf5dc3fedc2b71ba496083886d2b1e23b462
Successfully built sqlalchemy
Installing collected packages: sqlalchemy
Successfully installed sqlalchemy-1.2.1
```

## Connecting to the Database

For this tutorial we will create an in-memory database; this database will not be saved to disk and will be lost as soon as our program exits. See below for disk-backed databases and for patterns to make connecting easier in a big program.

In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)

`engine` is our database reference:

In [2]:
print(engine, type(engine))

Engine(sqlite:///:memory:) <class 'sqlalchemy.engine.base.Engine'>


We also need a database session to interact with:

In [3]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine, autocommit=True)
session = Session()

`session` is our starting point to initiate all database interactions:

In [4]:
print(session, type(session))

<sqlalchemy.orm.session.Session object at 0x11237c610> <class 'sqlalchemy.orm.session.Session'>


## Defining an Object Type

Objects are defined using classes (just like regular Python classes) that inherit from a `sqlalchemy` construct called a `declarative_base`:

In [5]:
from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy import Column
from sqlalchemy import Integer
from sqlalchemy import String

Base = declarative_base()

class Person(Base):
    
    __tablename__ = 'person'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String(25), nullable=False)
    last_name = Column(String(25), nullable=False)
    alter_ego = Column(String)


There are a few things going on here:

1. we create a base class called `Base` by calling `sqlalchemy.ext.declarative.declarative_base()` once. This `Base` will be the base class for all of our models (we don't have to create a new one for each model).
1. we create a class `Person` that inherits from `Base`
    1. we specify the name that should be used in the database to store objects of type `Person`
    1. we give the person an **primary key**, the authoritative way to reference the person throughout our program
    1. we define the columns that make up a `Person`, in this case first name, last name, and a way to store how we met the person

We can inspect the `Person` class to see how SQL Alchemy understands it:

In [6]:
Person.__table__

Table('person', MetaData(bind=None), Column('id', Integer(), table=<person>, primary_key=True, nullable=False), Column('first_name', String(length=25), table=<person>, nullable=False), Column('last_name', String(length=25), table=<person>, nullable=False), Column('alter_ego', String(), table=<person>), schema=None)

We also need to tell our database to create the table necessary to store `Person` objects:

In [7]:
Base.metadata.create_all(engine)

2019-07-09 16:19:35,574 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 16:19:35,575 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 16:19:35,577 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 16:19:35,578 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 16:19:35,579 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("person")
2019-07-09 16:19:35,580 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 16:19:35,582 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE person (
	id INTEGER NOT NULL, 
	first_name VARCHAR(25) NOT NULL, 
	last_name VARCHAR(25) NOT NULL, 
	alter_ego VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 16:19:35,582 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 16:19:35,584 INFO sqlalchemy.engine.base.Engine COMMIT


## Creating an Object

Now that we have our `Person` class, we can create objects fairly simply:

In [8]:
bruce_wayne = Person(first_name='Bruce', last_name='Wayne')
print(bruce_wayne.first_name)
print(bruce_wayne.id)

Bruce
None


Then we save it:

In [9]:
session.add(bruce_wayne)
session.flush()
print(bruce_wayne.id)

2019-07-09 16:19:35,597 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 16:19:35,599 INFO sqlalchemy.engine.base.Engine INSERT INTO person (first_name, last_name, alter_ego) VALUES (?, ?, ?)
2019-07-09 16:19:35,600 INFO sqlalchemy.engine.base.Engine ('Bruce', 'Wayne', None)
2019-07-09 16:19:35,603 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 16:19:35,605 INFO sqlalchemy.engine.base.Engine SELECT person.id AS person_id, person.first_name AS person_first_name, person.last_name AS person_last_name, person.alter_ego AS person_alter_ego 
FROM person 
WHERE person.id = ?
2019-07-09 16:19:35,606 INFO sqlalchemy.engine.base.Engine (1,)
1


Let's add a few more people:

In [10]:
session.add(Person(first_name='Tony', last_name='Stark'))
session.add(Person(first_name='Natasha', last_name='Romanova'))
session.flush()

2019-07-09 16:19:35,613 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 16:19:35,615 INFO sqlalchemy.engine.base.Engine INSERT INTO person (first_name, last_name, alter_ego) VALUES (?, ?, ?)
2019-07-09 16:19:35,616 INFO sqlalchemy.engine.base.Engine ('Tony', 'Stark', None)
2019-07-09 16:19:35,617 INFO sqlalchemy.engine.base.Engine INSERT INTO person (first_name, last_name, alter_ego) VALUES (?, ?, ?)
2019-07-09 16:19:35,618 INFO sqlalchemy.engine.base.Engine ('Natasha', 'Romanova', None)
2019-07-09 16:19:35,619 INFO sqlalchemy.engine.base.Engine COMMIT


**Note:** SQL Alchemy prevents me from creating objects that do not contain the required fields. You can see that I left out values for `alter_ego` above, but I cannot save an object that is missing a value for a field for which `nullable == False`:

In [11]:
# session.add(Person(first_name='Bob')) <- this is missing the last_name field
# session.flush() <- this will error because the previous object won't pass validation

## Retrieving Objects

Now that I have objects in my database, I can query and retrieve them:

In [12]:
people = session.query(Person).filter(Person.first_name=='Bruce').all()
print(len(people))
bruce1 = people[0]
print(bruce1.last_name)

2019-07-09 16:19:35,630 INFO sqlalchemy.engine.base.Engine SELECT person.id AS person_id, person.first_name AS person_first_name, person.last_name AS person_last_name, person.alter_ego AS person_alter_ego 
FROM person 
WHERE person.first_name = ?
2019-07-09 16:19:35,631 INFO sqlalchemy.engine.base.Engine ('Bruce',)
1
Wayne


What we just did:

1. queried the database for any `Person` object:
    1. that has the first name "Bruce"
    1. ... retrieve all of them
1. set `bruce1` to the value of the first entity retrieved

The `query` method is extremely powerful, allowing you to write just about any query you would want to retrieve whatever subset of your data you care about. For instance, to get *just* the first names of all people in the database:

In [13]:
for name in session.query(Person.first_name).all():
    print(name)

2019-07-09 16:19:35,637 INFO sqlalchemy.engine.base.Engine SELECT person.first_name AS person_first_name 
FROM person
2019-07-09 16:19:35,638 INFO sqlalchemy.engine.base.Engine ()
('Bruce',)
('Tony',)
('Natasha',)


## Updating Objects

Once you have retrieved the object you are interested in, you update it by changing the properties that need changing and re-saving:

In [14]:
bruce1.alter_ego = 'Batman'
print(session.dirty)
session.flush()
print(session.dirty)

IdentitySet([<__main__.Person object at 0x10bb62588>])
2019-07-09 16:19:35,647 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 16:19:35,650 INFO sqlalchemy.engine.base.Engine UPDATE person SET alter_ego=? WHERE person.id = ?
2019-07-09 16:19:35,650 INFO sqlalchemy.engine.base.Engine ('Batman', 1)
2019-07-09 16:19:35,651 INFO sqlalchemy.engine.base.Engine COMMIT
IdentitySet([])


Now we can query for all unique alter egos that we have identified:

In [15]:
for person in session.query(
        Person.alter_ego,
        Person.first_name,
        Person.last_name
        ).filter(Person.alter_ego != None).all():
    print(person)

2019-07-09 16:19:35,660 INFO sqlalchemy.engine.base.Engine SELECT person.alter_ego AS person_alter_ego, person.first_name AS person_first_name, person.last_name AS person_last_name 
FROM person 
WHERE person.alter_ego IS NOT NULL
2019-07-09 16:19:35,662 INFO sqlalchemy.engine.base.Engine ()
('Batman', 'Bruce', 'Wayne')


## Deleting Objects

Deleting objects is also simple:

In [16]:
print(session.query(Person).filter(Person.first_name=='Bruce').count())
session.delete(bruce1)
print(session.query(Person).filter(Person.first_name=='Bruce').count())

2019-07-09 16:19:35,672 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT person.id AS person_id, person.first_name AS person_first_name, person.last_name AS person_last_name, person.alter_ego AS person_alter_ego 
FROM person 
WHERE person.first_name = ?) AS anon_1
2019-07-09 16:19:35,673 INFO sqlalchemy.engine.base.Engine ('Bruce',)
1
2019-07-09 16:19:35,676 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 16:19:35,678 INFO sqlalchemy.engine.base.Engine SELECT person.id AS person_id, person.first_name AS person_first_name, person.last_name AS person_last_name, person.alter_ego AS person_alter_ego 
FROM person 
WHERE person.id = ?
2019-07-09 16:19:35,679 INFO sqlalchemy.engine.base.Engine (1,)
2019-07-09 16:19:35,680 INFO sqlalchemy.engine.base.Engine DELETE FROM person WHERE person.id = ?
2019-07-09 16:19:35,681 INFO sqlalchemy.engine.base.Engine (1,)
2019-07-09 16:19:35,683 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 16:19:35,684 INFO 

## Other Significant Concepts

We are skipping a *lot* of the meat of SQL Alchemy, but it has ways to handle all of the following database concepts:

* table relationships using foreign keys
* full transaction support
* persisting to many database backends, including MySQL and PostgreSQL
* ... and a whole lot more

## Saving Your Database to Disk

If you want your database saved to disk instead of being stored in memory, make the following change to the statement where we create the `engine` object:

```python
engine = create_engine('sqlite:///path/to/file', echo=True)
```

## Basic Program Pattern

The following template can be used to write programs using `sqlalchemy`:

### models.py

Create a file called `models.py` with the following:

```python
import os

from sqlalchemy import create_engine
from sqlalchemy import Boolean
from sqlalchemy import Column
from sqlalchemy import DateTime
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import UniqueConstraint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

BASE_DIR = os.path.basename(__name__)
FILENAME = 'database.sqlite'

db_name = os.path.join(BASE_DIR, FILENAME)

engine = create_engine('sqlite:///{}'.format(db_name))

Base = declarative_base()

#
# put your models here
#

# class MyModel...
```

### main.py

Create a file called `main.py` with the following:

```python
from sqlalchemy.orm import sessionmaker

from models import engine
from models import MyModel

with sessionmaker(bind=engine, autocommit=True)() as session:
    for person in session.query(Person).all():
        print(person)
    # put your fancy program stuff here
```

## References

* [SQL Alchemy Tutorial](https://docs.sqlalchemy.org/en/latest/orm/tutorial.html)
* [SQL Alchemy Documentation](https://docs.sqlalchemy.org)